### Config

In [10]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7,
)

In [5]:

# Fungsi tanpa tool
def ask(prompt: str):
    return llm.invoke([HumanMessage(content=prompt)]).content


### Tools

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain_community.tools.tavily_search import TavilySearchResults


# Fungsi dengan tool
def ask_with_search(prompt: str):
    # Setup search tool
    search_tool = TavilySearchResults()
    tools = [
        Tool(
            name="tavily-search",
            func=search_tool.run,
            description="Use this for answering questions about current events or factual information from the web."
        )
    ]

    # Setup agent
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

    # Jalankan pertanyaan dengan agen
    return agent.run(prompt)

In [12]:
# Pertanyaan biasa (tidak butuh search)
print(ask("Siapa presiden Indonesia sekarang?"))

Presiden Indonesia saat ini adalah Joko Widodo.


In [13]:
# Pertanyaan butuh pencarian web
print(ask_with_search("Siapa presiden Indonesia sekarang?"))



> Entering new AgentExecutor chain...
Pertanyaan ini membutuhkan informasi faktual terkini, jadi saya akan menggunakan alat pencarian.
Action: tavily-search
Action Input: "current president of Indonesia"
Observation: [{'title': 'Prabowo Subianto - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Prabowo_Subianto', 'content': "Prabowo Subianto Djojohadikusumo (EYD: Joyohadikusumo; born 17 October 1951) is an Indonesian politician, businessman, and former army general serving as the eighth president of Indonesia since 2024. He was previously the 26th minister of defense under president Joko Widodo from 2019 to 2024. Prabowo is Indonesia's third president to have a military background after Suharto and Susilo Bambang Yudhoyono and is the oldest first-term president in Indonesian history. [...] Political offices\nPreceded\xa0byJoko Widodo | President of Indonesia20 October 2024 – present | Incumbent\nPreceded\xa0byRyamizard Ryacudu | Minister of Defense2019–2024 | Succeeded\xa0bySjafrie

In [ ]:
## Tool From Scratch

import os
from datetime import datetime
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



# === TOOL 1: Calculator ===
@tool
def calculator_tool(expression: str) -> str:
    """Evaluasi ekspresi matematika seperti '12 / (2 + 1)'."""
    try:
        result = eval(expression, {"__builtins__": {}})
        return f"Hasil dari '{expression}' adalah {result}"
    except Exception as e:
        return f"Kesalahan menghitung ekspresi: {str(e)}"

# === TOOL 2: Time Now ===
@tool
def time_tool() -> str:
    """Ambil waktu dan tanggal terkini dalam format formal. """
    now = datetime.now()
    return now.strftime("Saat ini: %A, %d %B %Y pukul %H:%M:%S")


In [31]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Kamu adalah asisten pintar yang bisa menggunakan tools untuk membantu pengguna."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# === Daftar Tools ===
tools = [calculator_tool, time_tool]

# === Setup Agent ===
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# === Fungsi akhir untuk tanya dengan tools ===
def ask_with_tools(prompt: str):
    return agent_executor.invoke({""
    "input": prompt,
    "chat_history" : []
    })["output"]


In [32]:
print(ask_with_tools("Hitung 123 * (2 + 3)"))



> Entering new AgentExecutor chain...

Invoking: `calculator_tool` with `{'expression': '123 * (2 + 3)'}`


Hasil dari '123 * (2 + 3)' adalah 615123 * (2 + 3) adalah 615

> Finished chain.
123 * (2 + 3) adalah 615


In [33]:
print(ask_with_tools("Tanggal dan jam sekarang berapa?"))



> Entering new AgentExecutor chain...

Invoking: `time_tool` with `{}`


Saat ini: Tuesday, 22 April 2025 pukul 14:47:12Saat ini: Selasa, 22 April 2025 pukul 14:47:12.

> Finished chain.
Saat ini: Selasa, 22 April 2025 pukul 14:47:12.


In [48]:
## Tools dokter
doctor_db = [
    {
        "nama": "dr. Andi Wijaya",
        "spesialis": "anak",
        "jadwal": ["Senin", "Rabu", "Jumat"],
        "tersedia": True
    },
    {
        "nama": "dr. Sinta Dewi",
        "spesialis": "gigi",
        "jadwal": ["Selasa", "Kamis"],
        "tersedia": False
    },
    {
        "nama": "dr. Rudi Santoso",
        "spesialis": "mata",
        "jadwal": ["Senin", "Rabu"],
        "tersedia": True
    },
    {
        "nama": "dr. Budi Hartono",
        "spesialis": "umum",
        "jadwal": ["Senin", "Selasa", "Rabu", "Kamis"],
        "tersedia": True
    },
    {
        "nama": "dr. Lisa Marlina",
        "spesialis": "kulit",
        "jadwal": ["Jumat", "Sabtu"],
        "tersedia": False
    }
]


In [ ]:
from langchain_core.tools import tool

@tool
def doctor_lookup(spesialis: str, hari: str) -> str:
    """Cek apakah ada dokter dari spesialis tertentu yang tersedia di hari tertentu."""
    spesialis = spesialis.lower()
    hari = hari.capitalize()

    hasil = []
    for dokter in doctor_db:
        if spesialis in dokter["spesialis"].lower() and hari in dokter["jadwal"]:
            status = "tersedia" if dokter["tersedia"] else "tidak tersedia"
            hasil.append(f"{dokter['nama']} ({dokter['spesialis']}) tersedia hari {hari}: {status}")
    
    if not hasil:
        return f"Tidak ditemukan dokter spesialis {spesialis} yang praktek hari {hari}."
    return "\n".join(hasil)


In [64]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

tools = [doctor_lookup, time_tool]

prompt = ChatPromptTemplate.from_messages([
    ("system", "1. Jika pengguna tidak menyebutkan hari tertentu, tampilkan seluruh jadwal dokter yang tersedia, lengkap dengan nama dokter dan hari prakteknya. 2. Jika pengguna menyebutkan hari tertentu, tampilkan dokter yang praktek pada hari tersebut. 3. Selalu beri tahu hari ini hari apa sebelum memberikan jadwal dokter 4. Jika dibutuhkan untuk mendapatkan informasi tentang jadwal atau dokter, gunakan tools yang tersedia. 5.Jangan pernah bertanya ulang ke pengguna tentang hari — langsung beri informasi yang relevan."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [65]:
def ask_dokter(prompt: str):
    return agent_executor.invoke({
        "input": prompt,
        "chat_history": [],
    })["output"]


In [66]:
print(ask_dokter("kapan dokter gigi praktek ?"))



> Entering new AgentExecutor chain...

Invoking: `time_tool` with `{}`


Saat ini: Tuesday, 22 April 2025 pukul 15:08:06
Invoking: `doctor_lookup` with `{'hari': 'Selasa', 'spesialis': 'gigi'}`
responded: Hari ini hari Selasa. Berikut jadwal dokter gigi yang tersedia:

dr. Sinta Dewi (gigi) tersedia hari Selasa: tidak tersediaSaat ini hari Selasa, 22 April 2025. Dokter gigi yang tersedia pada hari Selasa adalah dr. Sinta Dewi, namun saat ini tidak tersedia.

> Finished chain.
Saat ini hari Selasa, 22 April 2025. Dokter gigi yang tersedia pada hari Selasa adalah dr. Sinta Dewi, namun saat ini tidak tersedia.


In [67]:
print(ask_dokter("Hari ini dokter umum tersedia enggak ?"))



> Entering new AgentExecutor chain...

Invoking: `time_tool` with `{}`


Saat ini: Tuesday, 22 April 2025 pukul 15:08:33
Invoking: `doctor_lookup` with `{'hari': 'Selasa', 'spesialis': 'umum'}`
responded: Hari ini Selasa, 22 April 2025.

dr. Budi Hartono (umum) tersedia hari Selasa: tersediaHari ini hari Selasa. dr. Budi Hartono (umum) tersedia hari ini.

> Finished chain.
Hari ini hari Selasa. dr. Budi Hartono (umum) tersedia hari ini.


### Tools Calling